Note that most of this notebook directly come from online source, the code layout is pretty bad, might take a while to understand what the original author is writing 

https://scikit-learn.org/stable/modules/classes.html#

In [1]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import random

In [8]:
# load data from a file and append it to the rawData
def loadGoldData(shuffle = True):
    loader = DataLoader()
    dec_data = loader.load_gold_data(_type = 'dec', neg_polarity=True, pos_polarity=True)
    truth_data = loader.load_gold_data(_type = 'truth', neg_polarity=True, pos_polarity=True)

    data = []

    for i in range(len(dec_data)):
        data.append((dec_data[i], 'fake'))
        data.append((truth_data[i], 'real'))

    if shuffle:
        random.shuffle(data)
    return data

# load data from a file and append it to the rawData
def loadData(path, Text=None):
    with open(path, encoding="utf8") as f:
        reader = csv.reader(f)

        next(reader)
        for line in reader:
            (Id, Rating, verified_Purchase, product_Category, Text, Label) = parseReview(line)
            rawData.append((Id, Rating, verified_Purchase, product_Category, Text, Label))
            #preprocessedData.append((Id, preProcess(Text), Label))
        

def splitData(percentage):
    dataSamples = len(rawData)
    halfOfData = int(len(rawData)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (_, Rating, verified_Purchase, product_Category, Text, Label) in rawData[:trainingSamples] + rawData[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text)),Label))
        # trainData.append((toFeatureVector(Rating, verified_Purchase, product_Category, preProcess(Text)),Label))
    for (_, Rating, verified_Purchase, product_Category, Text, Label) in rawData[trainingSamples:halfOfData] + rawData[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))
# QUESTION 1


# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
    # Should return a triple of an integer, a string containing the review, and a string indicating the label
    s=""
    if reviewLine[1]=="__label1__":
        s = "fake"
    else: 
        s = "real"
    return (reviewLine[0], reviewLine[2], reviewLine[3],reviewLine[4], reviewLine[8], s)
# TEXT PREPROCESSING AND FEATURE VECTORIZATION
# Input: a string of one review
table = str.maketrans({key: None for key in string.punctuation})


def preProcess(text):
    # Should return a list of tokens
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    lemmatized_tokens = []
    stop_words = set(stopwords.words('english'))
    text = text.translate(table)
    for w in text.split(" "):

        ######################## try without removing ################################################################
        lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))


        # if w not in stop_words:
        #     lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens
    return filtered_tokens
featureDict = {} # A global dictionary of features


# def toFeatureVector(Rating, verified_Purchase, product_Category, tokens):
def toFeatureVector(tokens):

    localDict = {}
    
# #Rating

#     #print(Rating)
#     featureDict["R"] = 1   
#     localDict["R"] = Rating

# #Verified_Purchase
  
#     featureDict["VP"] = 1
            
#     if verified_Purchase == "N":
#         localDict["VP"] = 0
#     else:
#         localDict["VP"] = 1

# #Product_Category

    
#     if product_Category not in featureDict:
#         featureDict[product_Category] = 1
#     else:
#         featureDict[product_Category] = +1
            
#     if product_Category not in localDict:
#         localDict[product_Category] = 1
#     else:
#         localDict[product_Category] = +1
            
            
#Text        

    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] = +1
            
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] = +1
    
    return localDict

In [4]:
import os
from utils.configs import config
from utils.loader import DataLoader



In [5]:
path = os.path.join(config['base_path'], config['amazon_path'], 'train')
reviewPath = DataLoader.list_all_txt_files(path)[0]
print(reviewPath)

data\amazon_reviews\train\amazon_reviews.txt


In [9]:
# MAIN

# loading reviews
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
path = os.path.join(config['base_path'], config['amazon_path'], 'train')
reviewPath = DataLoader.list_all_txt_files(path)[0]

## Do the actual stuff
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')

loadData(reviewPath) 


# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')

      
splitData(0.8)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')




Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 15750 rawData, 0 trainData, 0 testData
Preparing training and test data...
Now 15750 rawData, 12600 trainData, 3150 testData
Training Samples: 
12600
Features: 
365196


In [10]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)


def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))
    

def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
        classifier = trainClassifier(dataset[:i]+dataset[foldSize+i:])
        y_pred = predictLabels(dataset[i:i+foldSize],classifier)
        a = accuracy_score(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred)
        (p,r,f,_) = precision_recall_fscore_support(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred, average ='macro')
        #print(a,p,r,f)
        cv_results.append((a,p,r,f))
    cv_results = (np.mean(np.array(cv_results),axis=0))
    return cv_results

In [21]:
rawGold = loadGoldData(shuffle = True)

In [22]:
testGold = []
for (Text, Label) in rawGold:
    testGold.append((toFeatureVector(preProcess(Text)),Label))

In [42]:
####################################################
# run this to change the testData

# test_Data = testGold
test_Data = testData


####################################################

In [24]:
# additional classifiers
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

def MNB_Classifier(trainData):
    print("Training Classifier...")
    MNB_classifier = SklearnClassifier(MultinomialNB())
    return MNB_classifier.train(trainData)


def BernoulliNB_Classifier(trainData):
    print("Training Classifier...")
    BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
    return BernoulliNB_classifier.train(trainData)


def LogisticRegression_Classifier(trainData):
    print("Training Classifier...")
    LogisticRegression_classifier = SklearnClassifier(LogisticRegression(max_iter=1000))
    return LogisticRegression_classifier.train(trainData)

def SGDClassifier_Classifier(trainData):
    print("Training Classifier...")
    SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
    return SGDClassifier_classifier.train(trainData)


In [25]:
# print("Mean of cross-validations (Accuracy, Precision, Recall, Fscore): ", crossValidate(trainData, 10))


In [44]:
#  TEST DATA
classifier = trainClassifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.6615873015873016
Precision:  0.661758086996922
Recall:  0.6615873015873016
f1-score:  0.6615762510159062


In [45]:
#  TEST DATA
classifier = MNB_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.6866666666666666
Precision:  0.6873326413045806
Recall:  0.6866666666666666
f1-score:  0.6865491206125269


In [46]:
#  TEST DATA
classifier = BernoulliNB_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.5955555555555555
Precision:  0.6752718791877126
Recall:  0.5955555555555555
f1-score:  0.5475558477823235


In [29]:
#  TEST DATA
classifier = LogisticRegression_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.639375
Precision:  0.6420113067746717
Recall:  0.639375
f1-score:  0.637693530011746


In [30]:
#  TEST DATA
classifier = SGDClassifier_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.613125
Precision:  0.6185410284580334
Recall:  0.613125
f1-score:  0.6086549529212383


In [14]:
test_samples = testData
pred = predictLabels(test_samples, classifier)
summ = 0
for i in range(len(test_samples)):
    correct = np.where(test_samples[i][1] == pred[i], 1, 0)
    # print(f'True: {test_samples[i][1]}, Predict: {pred[i]},   {correct}')
    summ += correct

print(f'Acc: {summ/len(test_samples)}')


Acc: 0.68


In [15]:
type(trainData)

list

In [16]:
print(trainData[1][0], trainData[1][1])

{'i didnt': 1, 'didnt really': 1, 'really like': 1, 'like putting': 1, 'putting the': 1, 'the studio': 1, 'studio rta': 1, 'rta atower': 1, 'atower corner': 1, 'corner wood': 1, 'wood computer': 1, 'computer desk': 1, 'desk with': 1, 'with hutch': 1, 'hutch in': 1, 'in pewter': 1, 'pewter and': 1, 'and cherry': 1, 'cherry together': 1, 'together but': 1, 'but it': 1, 'it work': 1, 'work great': 1, 'great and': 1, 'and fit': 1, 'fit into': 1, 'into my': 1, 'my small': 1, 'small office': 1, 'office corner': 1, 'i': 1, 'didnt': 1, 'really': 1, 'like': 1, 'putting': 1, 'the': 1, 'studio': 1, 'rta': 1, 'atower': 1, 'corner': 1, 'wood': 1, 'computer': 1, 'desk': 1, 'with': 1, 'hutch': 1, 'in': 1, 'pewter': 1, 'and': 1, 'cherry': 1, 'together': 1, 'but': 1, 'it': 1, 'work': 1, 'great': 1, 'fit': 1, 'into': 1, 'my': 1, 'small': 1, 'office': 1} fake


In [17]:
type(trainData[1][0])

dict

In [ ]:
classifier = trainClassifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))

In [47]:
from sklearn.neural_network import MLPClassifier

def MLPClassifier_Classifier(trainData):
    print("Training Classifier...")
    MNB_classifier = SklearnClassifier(MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, verbose=True))
    return MNB_classifier.train(trainData)

#  TEST DATA
classifier = MLPClassifier_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
Iteration 1, loss = 0.60674646
Iteration 2, loss = 0.16915381
Iteration 3, loss = 0.03739610
Iteration 4, loss = 0.01190748
Iteration 5, loss = 0.00604867
Iteration 6, loss = 0.00385130
Iteration 7, loss = 0.00283067
Iteration 8, loss = 0.00226345
Iteration 9, loss = 0.00191653
Iteration 10, loss = 0.00169138
Iteration 11, loss = 0.00153529
Iteration 12, loss = 0.00142295
Iteration 13, loss = 0.00133724
Iteration 14, loss = 0.00126776
Iteration 15, loss = 0.00121308
Iteration 16, loss = 0.00116794
Iteration 17, loss = 0.00112959
Iteration 18, loss = 0.00109644
Iteration 19, loss = 0.00106716
Iteration 20, loss = 0.00104369
Iteration 21, loss = 0.00101888
Iteration 22, loss = 0.00099648
Iteration 23, loss = 0.00097614
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
accuracy:  0.665079365079365
Precision:  0.6654168958574547
Recall:  0.665079365079365
f1-score:  0.6650331499503432


In [43]:
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

accuracy:  0.6628571428571428
Precision:  0.6633553245863717
Recall:  0.6628571428571428
f1-score:  0.6627652423497419
